# Coordinates and TimeZones
With the saved countries dataset (my repo -> Python-Daily/Fun with Countries), we shall fetch the lat-long coordinates and timezones for our list of countries.

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import io

# Fetching our dataset from my Github repo (using the bytesIO process for a numpy-file case)
url = 'https://github.com/FaarisRazi/Python-Daily/blob/main/Fun%20with%20Countries/countries_geo.npy'
response = requests.get(url+'?raw=true')
response.raise_for_status()
bytesIO_obj = io.BytesIO(response.content)

country_df = np.load(bytesIO_obj, allow_pickle = True).item()['df']
country_df = country_df[['iso_a3']] # Just keeping the ISO-codes column

print(f'Head of our dataframe (total {country_df.shape[0]} rows/countries):')
country_df.head()

Head of our dataframe (total 255 rows/countries):


,iso_a3
country,
Aruba,ABW
Afghanistan,AFG
Angola,AGO
Anguilla,AIA
Albania,ALB


### Getting every Country's Coordinates
To geocode our country's addresses and obtain their lat-long coordinates, the **geocoder** API does the job!
*Source*: https://geocoder.readthedocs.io/api.html

Here's a test in fetching some random country's coordinates via ArcGIS in geocoder (Google is an option too).

In [2]:
from ttictoc import tic, toc # To track the time
from geocoder import arcgis
from random import choice

random_country = choice(country_df.index)

tic()
geo_obj =  arcgis(random_country)
coords = geo_obj.latlng

print(f"ArcGIS test for the Coordinates of {random_country} is:"+
      f"\nLatitude = {coords[0]}\nLongitude = {coords[1]}\n"+
      f"\n\nGeocoding time taken: {round(toc(), 3)} seconds.")

ArcGIS test for the Coordinates of Bajo Nuevo Bank (Petrel Is.) is:
Latitude = 15.839820000000032
Longitude = -78.67034999999998


Geocoding time taken: 0.962 seconds.


To geocode **multiple addresses** as in our case, we need "persistent HTTP connection" with requests.Session() as per the geocoder's API docs (see *Using a Session* section). Geocoding 255 countries may take approx. ~260 seconds given the time taken for one above. 

In [3]:
tic()

country_coords = {}
with requests.Session() as session:
    
    for i, country in enumerate(country_df.index, 1):
        coords = arcgis(country, session=session).latlng
        
        country_coords[country] = coords
        
        # For tracking progress,
        if not i%20: # print every 20th country's coordinates.
            print(f"{country}'s:\t{coords}")

print(f"\nAll {country_df.shape[0]} country's coordinates collected! That took {round(toc(),3)} seconds.")

Belgium's:	[50.640682937000065, 4.661070427000027]
Canada's:	[60.108670000000075, -113.64257999999995]
Cyprus's:	[35.05017420900003, 33.22622977900005]
Federated States of Micronesia's:	[6.880382388000044, 158.22751848500002]
Croatia's:	[45.11867958500005, 15.435623834000069]
Kazakhstan's:	[48.18310616400004, 67.19504548200007]
Macao S.A.R's:	[22.15778000000006, 113.55972000000008]
Malawi's:	[-13.50952235099993, 34.24073561300003]
Philippines's:	[14.164862797000069, 120.8616300000001]
Serranilla Bank's:	[15.84907000000004, -79.85961999999995]
Syria's:	[35.01280854600003, 38.50527333900004]
US Naval Base Guantanamo Bay's:	[20.48028000000005, -74.61693999999994]

All 255 country's coordinates collected! That took 149.695 seconds.


#### Coordinates in our dataframe:

In [4]:
# You can use our dictionary (country_coords) values instead of mapping below, but this felt safer:
country_df[['Latitude','Longitude']] = pd.DataFrame(country_df.index.map(country_coords.get).tolist(),
                                                   index = country_df.index)
country_df

,iso_a3,Latitude,Longitude
country,,,
Aruba,ABW,12.509085,-69.970503
Afghanistan,AFG,33.831137,66.024712
Angola,AGO,-12.293656,17.545335
Anguilla,AIA,18.224603,-63.059623
Albania,ALB,41.134553,20.064206
...,...,...,...
Samoa,WSM,-13.620808,-172.447333
Yemen,YEM,15.905206,47.593953
South Africa,ZAF,-28.997182,25.085050


### Getting every Country's TimeZone
Here's a function to collect each country's timezone, primarily using the *TimezoneFinder* package. (https://pypi.org/project/timezonefinder/).

In [5]:
from timezonefinder import TimezoneFinder

def gettz(coords):
    """
    Get the Timezone name of location coordinates via TimezoneFinder 
    API.

    Parameters
    ----------
    lat : float
        Latitude in degrees north
    lon : float
        Longitude in degrees east

    Returns
    -------
    Timezone: str
        TZ name of the location coordinates.
    """
    if type(coords) == str:
        coords = list(map(lambda x: float(x.strip()),coords.split(',')))

    lat, lon = coords

    tf = TimezoneFinder()

    try:
        timezone_name = tf.timezone_at(lng=lon, lat=lat)
        if timezone_name is None:
            timezone_name = tf.closest_timezone_at(lng=lon, lat=lat)

        return timezone_name
            # maybe even increase the search radius when it is still None
    except ValueError or NameError:
        # the coordinates were out of bounds
        pass # {handle error}

#### Timezones Collected:
Mapping our function above with each coordinate \[Latitude, Longitude\] value to get the country's timezone.

In [6]:
country_df['timezone'] = country_df.apply(lambda x: gettz([x['Latitude'], x['Longitude']]), 
                                          axis=1)
country_df

,iso_a3,Latitude,Longitude,timezone
country,,,,
Aruba,ABW,12.509085,-69.970503,America/Aruba
Afghanistan,AFG,33.831137,66.024712,Asia/Kabul
Angola,AGO,-12.293656,17.545335,Africa/Luanda
Anguilla,AIA,18.224603,-63.059623,America/Anguilla
Albania,ALB,41.134553,20.064206,Europe/Tirane
...,...,...,...,...
Samoa,WSM,-13.620808,-172.447333,Pacific/Apia
Yemen,YEM,15.905206,47.593953,Asia/Aden
South Africa,ZAF,-28.997182,25.085050,Africa/Johannesburg


In [8]:
# Lets save our dataframe via Numpy:
np.save('country_coords_tz', country_df, allow_pickle=True)

print('Dataframe saved!')

Dataframe saved!
